In [18]:
import numpy as np
import pandas as pd
import json

In [19]:
meta = pd.read_csv('dataset/movies_metadata.csv',low_memory=False)

meta.head()

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [20]:
meta = meta[['id','original_title',  'original_language',  'genres']]
meta = meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language'] == 'en']
meta.head()

movieId               original_title original_language  \
0     862                    Toy Story                en   
1    8844                      Jumanji                en   
2   15602             Grumpier Old Men                en   
3   31357            Waiting to Exhale                en   
4   11862  Father of the Bride Part II                en   

                                              genres  
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...  
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  
4                     [{'id': 35, 'name': 'Comedy'}]

In [21]:
ratings = pd.read_csv('dataset/ratings_small.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

userId  movieId  rating
0       1       31     2.5
1       1     1029     3.0
2       1     1061     3.0
3       1     1129     2.0
4       1     1172     4.0

In [22]:
ratings.describe()

userId        movieId         rating
count  100004.000000  100004.000000  100004.000000
mean      347.011310   12548.664363       3.543608
std       195.163838   26369.198969       1.058064
min         1.000000       1.000000       0.500000
25%       182.000000    1028.000000       3.000000
50%       367.000000    2406.500000       4.000000
75%       520.000000    5418.000000       4.000000
max       671.000000  163949.000000       5.000000

In [23]:
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')

In [24]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'','"'))
    gernes_list = []
    for g in genres:
        gernes_list.append(g['name'])
    return gernes_list

meta['genres'] = meta['genres'].apply(parse_genres)

In [25]:
meta.head()

movieId               original_title original_language  \
0      862                    Toy Story                en   
1     8844                      Jumanji                en   
2    15602             Grumpier Old Men                en   
3    31357            Waiting to Exhale                en   
4    11862  Father of the Bride Part II                en   

                         genres  
0   [Animation, Comedy, Family]  
1  [Adventure, Fantasy, Family]  
2             [Romance, Comedy]  
3      [Comedy, Drama, Romance]  
4                      [Comedy]

In [26]:
data = pd.merge(ratings, meta, on='movieId', how='inner')

data.head()

userId  movieId  rating original_title original_language   genres
0       1     1371     2.5      Rocky III                en  [Drama]
1       4     1371     4.0      Rocky III                en  [Drama]
2       7     1371     3.0      Rocky III                en  [Drama]
3      19     1371     4.0      Rocky III                en  [Drama]
4      21     1371     3.0      Rocky III                en  [Drama]

In [27]:
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

matrix.head()

original_title  !Women Art Revolution  'Gator Bait  \
userId                                               
1                                 NaN          NaN   
2                                 NaN          NaN   
3                                 NaN          NaN   
4                                 NaN          NaN   
5                                 NaN          NaN   

original_title  'Twas the Night Before Christmas  10 Items or Less  \
userId                                                               
1                                            NaN               NaN   
2                                            NaN               NaN   
3                                            NaN               NaN   
4                                            NaN               NaN   
5                                            NaN               NaN   

original_title  10 Things I Hate About You  10,000 BC  \
userId                                                  
1                                      NaN        NaN   
2                                      NaN        NaN   
3                                      NaN        NaN   
4                                      NaN        NaN   
5                                      NaN        NaN   

original_title  11'09''01 - September 11  12 + 1  12 Angry Men  1408  ...  \
userId                                                                ...   
1                                    NaN     NaN           NaN   NaN  ...   
2                                    NaN     NaN           NaN   NaN  ...   
3                                    NaN     NaN           NaN   NaN  ...   
4                                    NaN     NaN           NaN   NaN  ...   
5                                    NaN     NaN           NaN   NaN  ...   

original_title  Young and Innocent  Zaat  Zabriskie Point  Zapped Again!  \
userId                                                                     
1                              NaN   NaN              NaN            NaN   
2                              NaN   NaN              NaN            NaN   
3                              3.5   NaN              NaN            NaN   
4                              NaN   NaN              NaN            NaN   
5                              3.5   NaN              NaN            NaN   

original_title  Zardoz  Zodiac  eXistenZ  xXx  ¡Three Amigos!  \
userId                                                          
1                  NaN     NaN       NaN  NaN             NaN   
2                  NaN     NaN       NaN  NaN             NaN   
3                  NaN     NaN       NaN  NaN             NaN   
4                  NaN     NaN       NaN  NaN             NaN   
5                  NaN     NaN       NaN  NaN             NaN   

original_title  Мой сводный брат Франкенштейн  
userId                                         
1                                         NaN  
2                                         NaN  
3                                         NaN  
4                                         NaN  
5                                         NaN  

[5 rows x 2034 columns]

In [28]:
GENRE_WEIGHT = 0.1

def pearsonR(s1, s2):
   
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]

    result = []
    for title in matrix.columns:
        if title == input_movie:
            continue

        # rating comparison
        cor = pearsonR(matrix[input_movie], matrix[title])
        
        temp_genres = []
        # genre comparison
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]

            same_count = np.sum(np.isin(input_genres, temp_genres))
            cor += (GENRE_WEIGHT * same_count)
        
        if np.isnan(cor):
            continue
        else:
            result.append((title, '{:.2f}'.format(cor), temp_genres))
            
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

In [36]:
recommend_result = recommend('Best Seller', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

/var/folders/r1/h3gw0g2d3_lcm_11hmzmlchr0000gn/T/ipykernel_45033/2184702276.py:8: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))


Title Correlation  \
0                                       Enigma        0.79   
1                                       Shark!        0.72   
2                                 Shooting War        0.72   
3                                 Blue Thunder        0.71   
4                            Cassandra's Dream        0.68   
5                               The River Wild        0.68   
6                              The Dark Knight        0.67   
7                                 Big Bad Mama        0.66   
8  Yamakasi - Les samouraïs des temps modernes        0.66   
9                                      Bullitt        0.65   

                                               Genre  
0           [Mystery, Drama, Thriller, Romance, War]  
1                                 [Action, Thriller]  
2                [Documentary, Action, War, History]  
3  [Science Fiction, Action, Thriller, Crime, Drama]  
4                           [Crime, Drama, Thriller]  
5               [Action, Adventure, Crime, Thriller]  
6                   [Drama, Action, Crime, Thriller]  
7                     [Action, Comedy, Crime, Drama]  
8                             [Action, Crime, Drama]  
9                   [Action, Crime, Drama, Thriller]